In [ ]:
# slovar barv, ki jih bomo uporabili kasneje za risanje grafov
barve = {1 : "blue", 2 : "green", 3 : "red", 4 : "orange", 5 : "purple", 6 : "gold", 7 : "gray"} 

def pakirno_stevilo(G, max_colors):
    # Poišče pakirno število grafa G, št. barv <= max_colors
    
    V = G.vertices()
    D = G.distance_matrix()
    
    # Ustvarimo CLP
    p = MixedIntegerLinearProgram(maximization=False)
    x = p.new_variable(binary=True) # x_v,i -> vozlišče v je pobarvano z barvo i
    z = p.new_variable(nonnegative=True) # šteje koliko barv smo porabili (pakirno število)
    
    # minimiziramo število uporabljenih barv
    p.set_objective(z[0])
    
    # Omejitve:
    # 1) vsako vozlišče je pobarvano z natanko eno barvo
    for v in V:
        p.add_constraint(sum(x[v, i] for i in range(1, max_colors + 1)) == 1)
    
    # 2) če sta vozlišči oddaljeni <i+1 sta pobarvani z različno barvo
    for v in V:
        for u in V:
            for i in range(1, max_colors + 1):
                if (D[v][u] <= i) and (v != u):
                    p.add_constraint(x[v, i] + x[u, i] <= 1)
    
    # 3) barva vsakega vozlišča ni večja od pakirnega števila
    for v in V:
        for i in range(1, max_colors + 1):
            p.add_constraint(i * x[v, i] <= z[0])                

    # reši CLP
    p.solve()
    #[x_sol, z_sol] = p.get_values(x, z[0]) 

    # izpišemo le grafe z večjim pakirnim številom
    if p.get_values(z[0]) >= 4:
        # ustvari slovar, ključi so vozlišča, vrednosti so barve vozlišč
        barve_vozlisc = {v: next(i for i in range(1, max_colors + 1) if p.get_values(x[v, i]) > 0.5) for v in V}
        
        # ustvari slovar barv s katerim bomo pobarvali vozlišča
        slovar_barv = {}
        for v, b in barve_vozlisc.items():
            if barve[b] in slovar_barv.keys():
                slovar_barv[barve[b]].append(v)
            else:
                slovar_barv[barve[b]] = [v]

        # narišemo graf in izpišemo rezultat
        plt = G.plot(vertex_color=slovar_barv)
        plt.show()
        print(barve_vozlisc, f"Pakirno kromatično število: {p.get_values(z[0])}")
            
# če rabiš shranit grafe jih samo fukneš u seznam